# Association Rule for Store Dataset

## Load Data

Load data dan tunjukan 5 row pertamanya

Dataset :
https://raw.githubusercontent.com/GTerrence/dataset/main/Groceries%20data.csv

In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend

from sklearn.preprocessing import OneHotEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# load the data set ans show the first five transaction
url = "https://raw.githubusercontent.com/GTerrence/dataset/main/Groceries%20data.csv"
dfstore = pd.read_csv(url, sep=',')
dfstore.head(5)

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,1808,2015-07-21,tropical fruit,2015,7,21,1
1,2552,2015-05-01,whole milk,2015,5,1,4
2,2300,2015-09-19,pip fruit,2015,9,19,5
3,1187,2015-12-12,other vegetables,2015,12,12,5
4,3037,2015-01-02,whole milk,2015,1,2,4


<h2> EDA

In [ ]:
dfstore.describe()

,Member_number,year,month,day,day_of_week
count,38765.000000,38765.000000,38765.000000,38765.000000,38765.000000
mean,3003.641868,2014.528518,6.477570,15.753231,3.014498
std,1153.611031,0.499193,3.431561,8.801391,1.987669
min,1000.000000,2014.000000,1.000000,1.000000,0.000000
25%,2002.000000,2014.000000,4.000000,8.000000,1.000000
50%,3005.000000,2015.000000,6.000000,16.000000,3.000000
75%,4007.000000,2015.000000,9.000000,23.000000,5.000000
max,5000.000000,2015.000000,12.000000,31.000000,6.000000


In [33]:
dfstore.isna().any()

Member_number      False
Date               False
itemDescription    False
year               False
month              False
day                False
day_of_week        False
member_id          False
dtype: bool

<h2> Data Preprocessing

Ubahlah dataset di atas menjadi bentuk one hot encoding. Asumsikanlah bahwa pembelian barang yang dilakukan oleh Customer ID yang sama dalam hari yang sama terjadi dalam 1 transaksi yang sama (satu Order ID)


In [15]:
dfstore_group = dfstore.groupby('member_id')
dfstore_group.head(5)

,Member_number,Date,itemDescription,year,month,day,day_of_week,member_id
0,1808,2015-07-21,tropical fruit,2015,7,21,1,18082015-07-21
1,2552,2015-05-01,whole milk,2015,5,1,4,25522015-05-01
2,2300,2015-09-19,pip fruit,2015,9,19,5,23002015-09-19
3,1187,2015-12-12,other vegetables,2015,12,12,5,11872015-12-12
4,3037,2015-01-02,whole milk,2015,1,2,4,30372015-01-02
...,...,...,...,...,...,...,...,...
38760,4471,2014-08-10,sliced cheese,2014,8,10,6,44712014-08-10
38761,2022,2014-02-23,candy,2014,2,23,6,20222014-02-23
38762,1097,2014-04-16,cake bar,2014,4,16,2,10972014-04-16
38763,1510,2014-03-12,fruit/vegetable juice,2014,3,12,2,15102014-03-12


In [ ]:
product = set()
product = dfstore['itemDescription'].unique()
print(list(product))

['tropical fruit', 'whole milk', 'pip fruit', 'other vegetables', 'rolls/buns', 'pot plants', 'citrus fruit', 'beef', 'frankfurter', 'chicken', 'butter', 'fruit/vegetable juice', 'packaged fruit/vegetables', 'chocolate', 'specialty bar', 'butter milk', 'bottled water', 'yogurt', 'sausage', 'brown bread', 'hamburger meat', 'root vegetables', 'pork', 'pastry', 'canned beer', 'berries', 'coffee', 'misc. beverages', 'ham', 'turkey', 'curd cheese', 'red/blush wine', 'frozen potato products', 'flour', 'sugar', 'frozen meals', 'herbs', 'soda', 'detergent', 'grapes', 'processed cheese', 'fish', 'sparkling wine', 'newspapers', 'curd', 'pasta', 'popcorn', 'finished products', 'beverages', 'bottled beer', 'dessert', 'dog food', 'specialty chocolate', 'condensed milk', 'cleaner', 'white wine', 'meat', 'ice cream', 'hard cheese', 'cream cheese ', 'liquor', 'pickled vegetables', 'liquor (appetizer)', 'UHT-milk', 'candy', 'onions', 'hair spray', 'photo/film', 'domestic eggs', 'margarine', 'shopping b

In [16]:
# concatenate the string
dfstore['itemDescription'] = dfstore.groupby(['Member_number', 'Date'])['itemDescription'].transform(lambda x: ','.join(x))

# drop duplicate data
dfstore = dfstore.drop_duplicates()

In [34]:
dfstore.head()

,Member_number,Date,itemDescription,year,month,day,day_of_week,member_id
0,1808,2015-07-21,"tropical fruit,rolls/buns,candy",2015,7,21,1,18082015-07-21
1,2552,2015-05-01,"whole milk,tropical fruit,chocolate",2015,5,1,4,25522015-05-01
2,2300,2015-09-19,"pip fruit,other vegetables,flour",2015,9,19,5,23002015-09-19
3,1187,2015-12-12,"other vegetables,onions,shopping bags",2015,12,12,5,11872015-12-12
4,3037,2015-01-02,"whole milk,other vegetables,white bread",2015,1,2,4,30372015-01-02


In [19]:
data = list(dfstore["itemDescription"].apply(lambda x:x.split(",") ))

In [21]:
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)

df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [30]:
df =df.replace(True,1)
df = df.replace(False,0)

df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
14959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14961,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


## Apriori Algorithm

Gunakanlah algroitma apriori untuk mencari tahu produk mana yang sering dibeli secara bersama-sama (min support = 0.01)

In [31]:
from mlxtend.frequent_patterns import apriori, association_rules
freq_purchase = apriori(df, use_colnames=True, min_support=0.01)
freq_purchase

,support,itemsets
0,0.021386,(UHT-milk)
1,0.033950,(beef)
2,0.021787,(berries)
3,0.016574,(beverages)
4,0.045312,(bottled beer)
...,...,...
64,0.010559,"(rolls/buns, other vegetables)"
65,0.014837,"(whole milk, other vegetables)"
66,0.013968,"(rolls/buns, whole milk)"
67,0.011629,"(whole milk, soda)"


Berdasarkan hasil di atas, buatlah association rules nya dengan syarat min confidence = 0.1 

In [32]:
asrule = association_rules(freq_purchase, metric="confidence", min_threshold=0.1)
asrule.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(other vegetables),(whole milk),0.122101,0.157923,0.014837,0.121511,0.769430,-0.004446,0.958551
1,(rolls/buns),(whole milk),0.110005,0.157923,0.013968,0.126974,0.804028,-0.003404,0.964550
2,(soda),(whole milk),0.097106,0.157923,0.011629,0.119752,0.758296,-0.003707,0.956636
3,(yogurt),(whole milk),0.085879,0.157923,0.011161,0.129961,0.822940,-0.002401,0.967861
